In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#### Part 2

In [2]:
df = pd.read_parquet(r'N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2. Vendas\2.NFs_Venda.parquet')
df = df[df['Fat_Somado_n_considera'].isna()]

df_columns = df[['Data de Emissão','Núm. NF-e', 'BU Produtos', 'N1']]

In [3]:
condicao = df_columns['N1'] == 'Material Elétrico'

# Em seguida, você altera a coluna 'BU Produtos' para 'Neutro' nessas linhas
df_columns.loc[condicao, 'BU Produtos'] = 'Neutro'

In [4]:
condicao = df_columns['N1'] == 'Estrutura de Montagem'

# Em seguida, você altera a coluna 'BU Produtos' para 'Neutro' nessas linhas
df_columns.loc[condicao, 'BU Produtos'] = 'Neutro'

In [ ]:
df_columns['BU Produtos'].value_counts()

In [6]:
data_limite = pd.to_datetime('01/01/2022', format='%d/%m/%Y')
df_columns = df_columns[pd.to_datetime(df_columns['Data de Emissão'], format='%d/%m/%Y') > data_limite]

In [7]:
df = pd.get_dummies(df_columns, columns=['BU Produtos'], dtype=int, prefix_sep='', prefix='')
#df = df.drop(columns='Data de Emissão')
df = df.groupby('Núm. NF-e').agg({'Data de Emissão': 'first', 'GD': 'sum', 'NC': 'sum', 'OG': 'sum', 'PE': 'sum'}).reset_index()


# Função para determinar o valor da coluna 'BU'
def seleciona(row):
    if row['GD'] > 0 and (row['OG'] >= 0):
        return 'GD'
    elif (row['NC'] > 0 ) and (row['OG'] >= 0) :
        return 'NC'
    elif row['OG'] > 0 and row['GD'] == 0 and row['NC'] == 0 and row['PE'] == 0 :
        return 'OG'
    else:
        return '0'
    
    
def projeto_especial(row):
    if row['PE'] > 0 :
        return '1'
    
# Adicionando a coluna 'BU' ao DataFrame
df['BU'] = df.apply(seleciona, axis=1)

df['Proj_Especial'] = df.apply(projeto_especial, axis=1)




In [ ]:
df['Data de Emissão'].iloc[0]

In [9]:
df_final = df[df['Núm. NF-e'] != '']

In [10]:
df_final = df_final.rename(columns={'Núm. NF-e' : 'NF', 'Data de Emissão': 'Data' })

In [ ]:
df_final.head(5)

In [ ]:
df_final['Data'].min()

In [13]:
df_final.to_excel(r'N:\Drives compartilhados\22 - BI\6.Data_Engineering\1.Automacoes\3.Operacoes\1. Logistica\Dimensionalidade_BU\Frete_Dim_BU.xlsx', index = False)
df_final.to_excel(r'N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2.Frete\2.Frete_Dim_BU.xlsx')
#df_final.to_parquet(r'N:\Drives compartilhados\13 - CUSTOS E PRECIFICAÇÃO\02.Precificação\5. Acompanhamento de margem Frete Junho\2.Frete_Dim_BU.parquet')
